In [ ]:
import json
from bs4 import BeautifulSoup
import pandas as pd
from eurlex_api import *
from azure_api import *

url_api_AML5 ='https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=CELEX:02015L0849-20240709'
url_api_AML6 = 'https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=OJ:L_202401640'
url_api_R_AML6 = 'https://eur-lex.europa.eu/legal-content/FR/TXT/HTML/?uri=OJ:L_202401689'
api_result_AML5 = get_api_result(url_api_AML5)
api_result_AML6 = get_api_result(url_api_AML6)
api_result_R_AML6 = get_api_result(url_api_R_AML6)


/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eur-lex.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eur-lex.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eur-lex.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
for elem in api_result_AML5:
        soup = BeautifulSoup(elem['content'], 'html.parser')
        text = get_content_from_html(soup)
#display(HTML(elem['content']))
print(type(elem['content']))


<class 'str'>


In [ ]:


# Chaîne JSON
json_data = {
    "articles": []
}

# Parser le HTML
soup = BeautifulSoup(elem['content'], 'html.parser')

# Trouver toutes les subdivisions
articles = soup.find_all('div', class_='eli-subdivision')

data = []

for article in articles:
    # Vérifier si c'est un article (ID contient 'art_')
    article_id = article.get('id', '')
    if not article_id.startswith('art_'):
        continue
    print(article_id)
    json_data["articles"].append(json.loads(call_gpt(article.get_text())))
# Nom du fichier où vous voulez sauvegarder le JSON
fichier_json = "articles_AML5.json"

# Écriture dans le fichier
with open(fichier_json, "w", encoding="utf-8") as fichier:
    json.dump(json_data, fichier, ensure_ascii=False, indent=4)
    

art_1
art_2


KeyboardInterrupt: 

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd


# Parser le HTML
soup = BeautifulSoup(elem['content'], 'html.parser')

# Extraire le titre de l'article
article_titles = soup.find_all('p', class_='title-article-norm')

# Extraire les données des paragraphes
data = []
for article_title in article_titles:
    paragraphs = soup.find_all('div', class_='norm', recursive=True)

    for paragraph in paragraphs:
        paragraph_number_tag = paragraph.find('span', class_='no-parag')
        paragraph_number = paragraph_number_tag.text.strip() if paragraph_number_tag else None

        # Vérifier les sous-paragraphes
        sub_paragraphs = paragraph.find_all('div', class_='grid-container')
        if sub_paragraphs:
            for sub in sub_paragraphs:
                sub_number = sub.find('span').text.strip() if sub.find('span') else None
                sub_content = sub.find('div', class_='grid-list-column-2').text.strip() if sub.find('div', class_='grid-list-column-2') else None
                data.append([article_title, paragraph_number, sub_number, sub_content])
        else:
            # Aucun sous-paragraphe trouvé
            content = paragraph.find('div', class_='norm inline-element').text.strip() if paragraph.find('div', class_='norm inline-element') else None
            data.append([article_title, paragraph_number, None, content])

# Convertir en DataFrame pour un affichage structuré
df = pd.DataFrame(data, columns=['Article', 'Paragraphe', 'Sous-paragraphe', 'Contenu'])

# Afficher le tableau
print(df.dropna(subset='Paragraphe'))

In [ ]:
df.to_csv('test.csv')